Connected to Python 3.10.11

In [2]:
import datetime

now = datetime.datetime.now()
print("Current date and time:", now)


Current date and time: 2025-06-30 13:17:51.326162


### Load libraries

In [1]:
import sys
import os
import itertools
import matplotlib.pyplot as plt
import pandas as pd

# Set paths
notebook_dir = os.getcwd()
src_path = r"C:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\src"
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import add_src_to_path
add_src_to_path()

from data_split import split_time_series_by_river
from rf_model import train_and_apply_rf_with_tuning
from plot_predictions import plot_predictions_by_river
from plot_predictions import plot_actual_vs_predicted

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

# Experiment parameters
river_systems = ["Fraser River", "Bristol Bay", "Columbia River"]
top_k_options = [6, 10, 0]  # 0 means use all features
add_feat_options = [True, False]
arima_options = [True, False]

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_path = os.path.join(project_root, 'data', 'Combined_FeatureSet_For_Model.csv')

model_list = ["RF", "GBRT", "XGB", "LR", "PR"]
all_experiments = []

for river_system, top_k_features, add_additional_features, use_arima_on_top in itertools.product(
    river_systems, top_k_options, add_feat_options, arima_options
):
    print(f"\n🧪 Running: {river_system} | TopK={top_k_features} | ExtraFeat={add_additional_features} | ARIMA={use_arima_on_top}")

    combined_df = pd.read_csv(data_path)

    columns_to_drop = [
    'mean_temp_JunAug', 'mean_temp_MaySep', 'max_temp_JunAug',
    'mean_discharge_MarMay', 'max_discharge_AprSep'
]
    combined_df = combined_df.drop(columns=[col for col in columns_to_drop if col in combined_df.columns])
    combined_df = combined_df[combined_df["System"] == river_system]

    if add_additional_features:
        features_to_lag = [
            'Total_Returns', 'AgeClass_0.1', 'AgeClass_0.2', 'AgeClass_0.3', 'AgeClass_0.4', 'AgeClass_0.5',
            'AgeClass_1.1', 'AgeClass_1.2', 'AgeClass_1.3', 'AgeClass_1.4', 'AgeClass_1.5',
            'AgeClass_2.1', 'AgeClass_2.2', 'AgeClass_2.3', 'AgeClass_2.4',
            'AgeClass_3.1', 'AgeClass_3.2', 'AgeClass_3.3', 'AgeClass_3.4',
            'Total_Returns_NextYear', 'Pacea_ALPI_Anomaly', 'npi_mean_NovMar', 'oni_mean_DecFeb',
            'npgo_mean_DecFeb', 'ao_mean_DecMar', 'pdo_mean_DecMar', 'pdo_mean_MaySep'
        ]
        for feat in features_to_lag:
            for lag in [1, 2, 3, 4, 5]:
                combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)

    if river_system == "Bristol Bay":
        combined_df = combined_df[combined_df["Year"] >= 1995]

    if river_system in ["Columbia River", "Bristol Bay"] and add_additional_features:
        if 'total_spawners_y_minus_2_to_4' in combined_df.columns:
            combined_df = combined_df.dropna(subset=['total_spawners_y_minus_2_to_4'])
        if 'AgeClass_0.2_Yminus5' in combined_df.columns:
            combined_df = combined_df.dropna(subset=['AgeClass_0.2_Yminus5'])

    combined_df = combined_df.dropna(axis=1, how='any').dropna()

    train_df, test_df = split_time_series_by_river(
        combined_df,
        time_column="Year",
        group_columns=["System", "River"],
        test_fraction=0.2,
        gap_years=0
    )

    train_df["River_Name"] = train_df["River"]
    test_df["River_Name"] = test_df["River"]

    train_df_encoded = pd.get_dummies(train_df, columns=["River"], prefix="River")
    test_df_encoded = pd.get_dummies(test_df, columns=["River"], prefix="River")

    for model_name in model_list:
        try:
            results = train_and_apply_rf_with_tuning(
                model=model_name,
                train_df=train_df_encoded,
                test_df=test_df_encoded,
                topk_feat=top_k_features
            )

            # ✅ Apply ARIMA correction if selected
            if use_arima_on_top:
                residuals = results["Timeline_train"]["Actual"] - results["Timeline_train"]["Predicted"]
                residuals_series = pd.Series(residuals.values, index=results["Timeline_train"]["Year"])

                arima_model = ARIMA(residuals_series, order=(1, 0, 0))
                arima_fit = arima_model.fit()

                residual_forecast = arima_fit.forecast(steps=len(results["Timeline_test"]["Predicted"]))
                hybrid_pred = results["Timeline_test"]["Predicted"] + residual_forecast.values

                results["Timeline_test"]["Predicted"] = hybrid_pred

                # Optionally update metrics
                grouped = results["Timeline_test"].groupby("River_Name")
                metrics_by_river_test = grouped.apply(
                    lambda g: pd.Series({
                        "R2": r2_score(g["Actual"], g["Predicted"]),
                        "MSE": mean_squared_error(g["Actual"], g["Predicted"]),
                        "MAPE": mean_absolute_percentage_error(g["Actual"], g["Predicted"]) * 100
                    })
                ).reset_index()

                results["Metrics_by_River_Test"] = metrics_by_river_test

            # 📊 Prepare final output
            test_df_ = results['Metrics_by_River_Test'].copy()
            train_df_ = results['Metrics_by_River_Train'].copy()

            test_df_ = test_df_.rename(columns={"R2": "R2_Test", "MSE": "MSE_Test", "MAPE": "MAPE_Test"})
            train_df_ = train_df_.rename(columns={"R2": "R2_Train", "MSE": "MSE_Train", "MAPE": "MAPE_Train"})

            merged_df = pd.merge(test_df_, train_df_, on="River_Name", how="outer")
            merged_df.insert(0, "Model", model_name)

            river_system_lookup = pd.concat([train_df_encoded, test_df_encoded])["River_Name"].drop_duplicates()
            river_system_lookup = pd.merge(
                river_system_lookup.to_frame(),
                combined_df[["River", "System"]].drop_duplicates(),
                left_on="River_Name",
                right_on="River",
                how="left"
            ).drop(columns="River")

            merged_df = pd.merge(merged_df, river_system_lookup, on="River_Name", how="left")

            merged_df["Selected_System"] = river_system
            merged_df["TopK_Features"] = top_k_features
            merged_df["Additional_Features_Used"] = add_additional_features
            merged_df["ARIMA_Enabled"] = use_arima_on_top

            feature_names = results.get("Selected_Feature_Names", [])
            merged_df["Num_Features_Used"] = len(feature_names)
            merged_df["Feature_Names"] = ", ".join(feature_names)
            print(merged_df["Feature_Names"])

            all_experiments.append(merged_df)

        except Exception as e:
            print(f"❌ Error with model {model_name}: {e}")

# 📝 Save final results
final_df = pd.concat(all_experiments, ignore_index=True)
output_path = r"C:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\results.csv"
final_df.to_csv(output_path, index=False)

print(f"\n✅ All experiment results saved to:\n{output_path}")


🧪 Running: Fraser River | TopK=6 | ExtraFeat=True | ARIMA=True
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'River_Chilko',
       'River_Quesnel', 'River_Raft'],
      dtype='object')


C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_13712\3760671169.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)
C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_13712\3760671169.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)
C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_13712\3760671169.py:61: PerformanceWarning: DataFra

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
RF R2: 0.41
RF MSE: 365025695484.21
RF MAPE: 477.37
0    Total_Returns, AgeClass_0.2, AgeClass_1.1, Riv...
1    Total_Returns, AgeClass_0.2, AgeClass_1.1, Riv...
2    Total_Returns, AgeClass_0.2, AgeClass_1.1, Riv...
3    Total_Returns, AgeClass_0.2, AgeClass_1.1, Riv...
4    Total_Returns, AgeClass_0.2, AgeClass_1.1, Riv...
Name: Feature_Names, dtype: object
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'River_Chilko',
       'River_Quesnel', 'River_Raft'],
      dtype='object')


c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:142: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  system_metrics_test = results_df.groupby("System", group_keys=False).apply(
c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:146: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  river_metrics_test = results_df.groupby("River_N

Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'max_iter': 200, 'min_samples_leaf': 1}
GBRT R2: 0.21
GBRT MSE: 486627008236.27
GBRT MAPE: 442.80
0    Total_Returns, AgeClass_0.2, AgeClass_1.1, Riv...
1    Total_Returns, AgeClass_0.2, AgeClass_1.1, Riv...
2    Total_Returns, AgeClass_0.2, AgeClass_1.1, Riv...
3    Total_Returns, AgeClass_0.2, AgeClass_1.1, Riv...
4    Total_Returns, AgeClass_0.2, AgeClass_1.1, Riv...
Name: Feature_Names, dtype: object
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'River_Chilko',
       'River_Quesnel', 'River_Raft'],
      dtype='object')


...

c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:142: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  system_metrics_test = results_df.groupby("System", group_keys=False).apply(
c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:146: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  river_metrics_test = results_df.groupby("River_N

Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'max_iter': 200, 'min_samples_leaf': 2}
GBRT R2: 0.65
GBRT MSE: 20461356112.79
GBRT MAPE: 72.54
0    Total_Returns, AgeClass_0.1, AgeClass_0.2, Age...
Name: Feature_Names, dtype: object
Selected features:
Index(['Total_Returns', 'AgeClass_0.1', 'AgeClass_0.2', 'AgeClass_0.3',
       'AgeClass_0.4', 'AgeClass_0.5', 'AgeClass_1.1', 'AgeClass_1.2',
       'AgeClass_1.3', 'AgeClass_1.4', 'AgeClass_1.5', 'AgeClass_2.1',
       'AgeClass_2.2', 'AgeClass_2.3', 'AgeClass_2.4', 'AgeClass_3.1',
       'AgeClass_3.2', 'AgeClass_3.3', 'AgeClass_3.4', 'npi_mean_NovMar',
       'oni_mean_DecFeb', 'mei_mean_AprSep', 'npgo_mean_DecFeb',
       'ao_mean_DecMar', 'pdo_mean_DecMar', 'pdo_mean_MaySep', 'sst_aprjul',
       'sst_anom', 'River_Bonneville Lock & Dam'],
      dtype='object')


c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:142: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  system_metrics_test = results_df.groupby("System", group_keys=False).apply(
c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:146: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  river_metrics_test = results_df.groupby("River_N

Best Parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200}
XGB R2: 0.21
XGB MSE: 46559114677.27
XGB MAPE: 85.72
0    Total_Returns, AgeClass_0.1, AgeClass_0.2, Age...
Name: Feature_Names, dtype: object
Selected features:
Index(['Total_Returns', 'AgeClass_0.1', 'AgeClass_0.2', 'AgeClass_0.3',
       'AgeClass_0.4', 'AgeClass_0.5', 'AgeClass_1.1', 'AgeClass_1.2',
       'AgeClass_1.3', 'AgeClass_1.4', 'AgeClass_1.5', 'AgeClass_2.1',
       'AgeClass_2.2', 'AgeClass_2.3', 'AgeClass_2.4', 'AgeClass_3.1',
       'AgeClass_3.2', 'AgeClass_3.3', 'AgeClass_3.4', 'npi_mean_NovMar',
       'oni_mean_DecFeb', 'mei_mean_AprSep', 'npgo_mean_DecFeb',
       'ao_mean_DecMar', 'pdo_mean_DecMar', 'pdo_mean_MaySep', 'sst_aprjul',
       'sst_anom', 'River_Bonneville Lock & Dam'],
      dtype='object')
Best Parameters: {'fit_intercept': False, 'positive': False}
LR R2: 0.51
LR MSE: 29015283255.17
LR MAPE: 82.04
0    Total_Returns, AgeClass_0.1, AgeClass_0.2, Age...
Name: Feature_Names,

c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:142: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  system_metrics_test = results_df.groupby("System", group_keys=False).apply(
c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:146: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  river_metrics_test = results_df.groupby("River_N